# Pairgrid Example

This notebook shows how to perform the data transformation needing to construct a standard pairgrid plot using Altair. This example illustrates one of the core ideas of Altair and vega-lite: *subplot facetting is equivalent to a groupby operation*. 

## Setup

In [ ]:
import numpy as np
import pandas as pd
import altair.api as alt

In [ ]:
alt.use_renderer('lightning')

## Load and transform data

In [ ]:
import seaborn as sns
iris = sns.load_dataset('iris')
iris.head()

Here is a function that transforms the data so that the pairgrid can be constructed using groupby operations:

In [ ]:
def pairgrid_transform(data, rows, cols=None):
    """
    Transform a dataframe to allow vega to produce a PairGrid
    
    """
    if cols is None:
        cols = rows
      
    M1 = len(rows)
    M2 = len(cols)
    N = len(data)
    
    data = pd.concat(M1 * M2 * [data])
    i = np.arange(N * M1 * M2)
    irow = np.arange(N * M1 * M2) // N % M1
    icol = np.arange(N * M1 * M2) // N // M1
    
    data['x'] = data[cols].values[i, icol]
    data['y'] = data[rows].values[i, irow]
    data['row'] = rows.values[irow]
    data['col'] = cols.values[icol]
    
    return data

Here is the transformed data. Notic how we now have `row` and `col` columns that contains the values we will groupby for the subplot facets:

In [ ]:
data = pairgrid_transform(iris, iris.columns[:4])
data.head()

## Visualization

Now the pairgrid is a simple encoding of the transformed data:

In [ ]:
v = alt.Viz(data).encode(x='x', y='y', row='row', col='col', color='species').point()
v.render()